# <font color='blue'>Files -- astropy.io.fits</font>

<font color='blue'>As an astronomer there may be a point where you have to read and write fits files. Thankfully, there's a convinent set of functions in the `fits` module of `astropy.io` that handles most types of fits files (such as images, cubes, multi-extension, binary tables). In this last section will review some of the basic functions within this module, and how to work with the most common data types. Unfortunately, as you haven't done plotting yet, we won't be able to display the images. More on that later!</font><br><br>

## <font color='blue'>Reading</font>

<font color='blue'>Let's start by reading in a few example files to see the different sorts it can read in. We'll start by using the `fits.open()` function, although there are some convinence functions that you can use to access image quickly that we will cover at later. Opening the file and using the `.info()` method on the result tells us about the different extensions, the lengths of their header, and the type and format of the data. We'll use the same `with` syntax as before so that we don't have to worry about closing it again.</font>

In [ ]:
from astropy.io import fits
with fits.open('test-im1.fits') as data:
    data.info() # This is a HDUList object

<font color='blue'>Here we have a very simple fits file. There is only the one extension. It is a 20x10 array of doubles (float64), and there is a very small header with six cards.<br><br>
To access the data we simply index the `HDUList` object (`data` in this case) with the appropriate index:</font>

In [ ]:
with fits.open('test-im1.fits') as data:
    print(data[0])

<font color='blue'>This is a `PrimaryHDU` object (we'll see more of this when we are writing a file later). It has the `data` and `header` attribute that let us see the image and header of this particular extension. Note that the first extension within any fits image is a `PrimaryHDU` object, the rest are typically `ImageHDU` objects.<br><br>

Let's print a subset of the image, and the header (using the built-in `repr` function for nice formatting)</font>

In [ ]:
with fits.open('test-im1.fits') as data:
    print(data[0].data[0:2,0:2])
    print(repr(data[0].header))

<font color='blue'>That wasn't so difficult. But if you wanted it to be even easier, we can use the convinience functions `fits.getdata()`, and `fits.getheader()`:</font>

In [ ]:
data = fits.getdata('test-im1.fits')
hdr = fits.getheader('test-im1.fits')
print(data[0:2,0:2])
print(repr(hdr))

<font color='blue'>You can access the individual keyword values of the header by indexing it with the appropriate keyword</font>

In [ ]:
print(hdr['BITPIX'], hdr['NAXIS1'], hdr['NAXIS2'])

<font color='blue'>and we can print a list of the header keywords using the `.keys()` method</font>

In [ ]:
print(list(hdr.keys()))

<font color='blue'>Of course, the data need not be a 2-d image, we are only limited by the number of dimensions supported by the fits protocol. Here's an example with a 3-d cube. Remember that numpy is row-major, so transposed relative to an IDL or DS9 image.</font>

In [ ]:
with fits.open('test-im2.fits') as data:
    data.info() # This is a HDUList object

In [ ]:
with fits.open('test-im2.fits') as data:
    print('Dimensions: ', data[0].data.shape)
    print(data[0].data[0:2,0:2,0:2])
    print(repr(data[0].header))

<font color='blue'>Observatories tend to supply observations within multi-extension fits files. There may be one extension per detector, or an extension for a data quality frame, for example.</font>

In [ ]:
with fits.open('test-im3.fits') as data:
    data.info()

<font color='blue'>In this file we have four extensions. The first extension (always a `PrimaryHDU` object) in this case is empty, but it does have an associated header. Some instruments may provide a blank first extension to store values from the telescope in the header, for example. The three subsequent extensions are all `ImageHDU` objects, or n-dimensional numerical arrays. These don't have to have the same dimensions or the same data type, and they each have their own header.<br><br>

You can address the extensions by index number, or by the name of the extension:</font>

In [ ]:
with fits.open('test-im3.fits') as data:
    print(data[1].data[0:2,0:2])
    print(data['SCI'].data[0:2,0:2])

<font color='blue'>and we can again use the two convinience functions to easily access the data and header from any extension using the `ext` keyword. This can either be the index of the extension or the name. Be careful if you have multiple extensions of the same name, it will return the first one unless the `extver` keyword is also set to something other than `1`.</font>

In [ ]:
data = fits.getdata('test-im3.fits', ext=1)
hdr = fits.getheader('test-im3.fits', ext=1)
print(data[0:2,0:2])
print(repr(hdr))

<font color='blue'>The last type of extensions we'll cover are binary tables. I think some ESO instruments produce fits files with this type of extension.</font>

In [ ]:
with fits.open('test-im4.fits') as data:
    data.info()

<font color='blue'>Here we have a blank first extension, and image extension, then a binary extension containing two rows and three columns. The format codes for the binary table are those used in the fits standard (`D` is double). We can use the `.columns` attribute to see the name and format of each column stored in the table</font>

In [ ]:
with fits.open('test-im4.fits') as data:
    print(data['ENV'].columns)

<font color='blue'>and we can access the individual columns using their name as an index:</font>

In [ ]:
with fits.open('test-im4.fits') as data:
    print(data['ENV'].data['dimm'])
    print(data['ENV'].data['mass'])

## <font color='blue'>Writing</font><br>
<font color='blue'>Writing images and data cubes to fits file is relatively straight forward. It's a bit more complicated if you want to save a multi-extension fits file, and I'll leave writing a binary table as an exercise for the reader!<br><br>

Let's start with a simple 2-d array. As with reading a file, there are two ways to do this. We'll start with the more formal way, which provides greatest flexibility, and then I'll highlight the convinence function later.</font>

In [ ]:
import numpy as np
array = np.random.uniform(size=(20, 10)) # Random numbers between 0-1
print(array[0:2,0:2])

<font color='blue'>First, we need to create a `PrimaryHDU` object within which to store our array. Remember that the first extension is always a `PrimaryHDU` object. If you don't have one of these as the first extension, it will return an error.</font>

In [ ]:
hdu0 = fits.ImageHDU(data=array)
hdu_list = fits.HDUList([hdu0])
hdu_list.writeto('my-new-im1.fits')

In [ ]:
hdu0 = fits.PrimaryHDU(data=array) # or fits.PrimaryHDU(array)
hdu_list = fits.HDUList([hdu0])
hdu_list.writeto('my-new-im1.fits', overwrite=True)

<font color='blue'>Let's open it as we did before to see if our array is saved correctly.</font>

In [ ]:
with fits.open('my-new-im1.fits') as data:
    data.info()

<font color='blue'>Even without specifying a header, a minimal header is included with all of the extensions. These are required by the fits standard</font>

In [ ]:
with fits.open('my-new-im1.fits') as data:
    print(repr(data[0].header))

<font color='blue'>If we want to add a header, we can make one using the `fits.Header()` object and add whatever keywords we like</font>

In [ ]:
hdr = fits.Header() # Makes a blank header
hdr['CREATED'] = '2020-04-23'
hdr['VERSION'] = '1.0'
print(repr(hdr))

<font color='blue'>Let's save it again...</font>

In [ ]:
hdu0 = fits.PrimaryHDU(data=array, header=hdr)
hdu_list = fits.HDUList([hdu0])
hdu_list.writeto('my-new-im1.fits', overwrite=True)

<font color='blue'>You can see that it has automatically added the keywords required by the fits standard:</font>

In [ ]:
with fits.open('my-new-im1.fits') as data:
    print(repr(data[0].header))

<font color='blue'>To add an additional extension, include it in your call to `fits.HDUList`</font>

In [ ]:
hdu1 = fits.ImageHDU(data=array, name='DQ')
hdu_list = fits.HDUList([hdu0, hdu1]) # Using hdu0 from above
hdu_list.writeto('my-new-im2.fits', overwrite=True)

In [ ]:
with fits.open('my-new-im2.fits') as data:
    print(data.info())

<font color='blue'>If you just want to write a single array to a fits file, the easiest way is via the convenience function `fits.writeto()`:</font>

In [ ]:
fits.writeto('my-new-im3.fits', array, overwrite=True, header=hdr) # Use hdr from above

***

<font color='blue'>The last thing we'll cover is the ability to read a fits file and update it without saving it to a new file. This can be useful if you want to update one particular extension in a file, for example. When you call `fits.open()` use the keyword `mode='update'` (the default behaviour is to open it in read-only mode).</font>

In [ ]:
with fits.open('my-new-im3.fits', mode='update') as hdu_list:
    print(hdu_list.info())
    print(hdu_list[0].data[0,0])
    
    hdu_list[0].data[0,0] = -99
    
    print(hdu_list[0].data[0,0])
    
    hdu_list.flush()
    

<font color='blue'>If we open the file again, we should see that our change was saved successfully.</font>

In [ ]:
with fits.open('my-new-im3.fits') as hdu_list:
    print(hdu_list[0].data[0,0])

## <font color='green'>Exercises</font>

<font color='green'>Write the following 2-d numpy array to a new file. If you are repeating this exercise, you may need to use the `overwrite=True` keyword so that it overwrites the existing file. Practice using the longer approach that uses a `HDUList` object, and using the convenience function `fits.writeto()` </font>

In [ ]:
import numpy as np
array = np.random.normal(size=(100, 100))


<font color='green'>Create a new header objecr `fits.Header()` and populate it with a few keyword/value pairs of your choice. Using the code from the previous exercise, write a new file with this header.</font>

<font color='green'>Open the fits file `j8eq51dnq_drz.fits` and use the `.info()` method to see how many extensions there are, and what type they are. Try printing the header and/or a (small) subset of the data from each of the extensions. For the last extension, the `BinTableHDU`, you can print the columns names by using the `.column` attribute (i.e. `data['HDRTAB'].column`), and you can print the single row in the table using the `.data` attribute.</font>

<font color='green'>Open `j8eq51dnq_drz.fits` again, assign the data and header from the `SCI` extension to new variables using the `.data` and `.header` attributes. Write the resulting array and header to a new fits file using either of the methods described previously.